In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import re
import pprint
from BCBio.GFF import GFFExaminer
from BCBio import GFF
from sklearn.metrics import r2_score

In [ ]:
names_of_scaffolds = []
for i in range(1, 26):
    names_of_scaffolds.append('scaffold_' + str(i)) 

In [ ]:
with open('ref_' + 'A10' + '.maf') as fin:
    fin.seek(0) #395
    data = fin.read()
data = data.split('\n')

In [ ]:
def inversion_search(datum):
    inv = {}
    for sc in datum.keys():
        data = datum[sc]
        inv[sc] = []
        used_bps = []
        for i in range(len(data)):
            for j in range(len(data)):
                if data[i][0] == data[j][0]:
                    if data[i][3] != data[j][3]:
                        #print('inv')
                        if (abs(data[i][4] - data[j][5]) < 30) or (abs(data[j][4] - data[i][5]) < 30):
                            if (abs(data[i][1] - data[j][2]) < 30) or (abs(data[j][1] - data[i][2]) < 30):
                                if (data[i][-1] > 30) and (data[j][-1] > 30):
                                    if (data[i][1] < data[j][1]):
                                        if data[i] not in used_bps and data[j] not in used_bps:
                                            already_in_list = 0
                                            for x in inv[sc]:
                                                #print(x, data[i])
                                                if abs(x[0][1] - data[i][1]) < 30 or abs(x[1][1] - data[j][1]) < 30:
                                                    already_in_list = 1
                                            if already_in_list == 0:
                                                inv[sc].append([data[i], data[j]])
                                            used_bps.append(data[i])
                                            used_bps.append(data[j])
    return inv

In [ ]:

def get_data(mushroom_name):
    names_of_scaffolds = []
    for i in range(1, 26):
        names_of_scaffolds.append('scaffold_' + str(i)) 


    filename = 'ref_' + mushroom_name + '.maf'
    with open(filename) as fin:
        fin.seek(0) #395
        data = fin.read()
    data = data.split('\n')
    for i in range(len(data)):
        del data[0]
        if data[0] == '':
            del data[0]
            break

    lines = []
    pos_start = 0
    for x in data:
        lines.append(x.split())
    good_lines = {}
    lines = [x for x in lines if x != []] 
    for i in range(len(lines)):
        if lines[i][0] == 'a':
            scaf_name = lines[i+1][1]
            pos_start, length = lines[i+1][2], lines[i+1][3]
            if scaf_name not in good_lines.keys():
                good_lines[scaf_name] = []
        else:
            lines[i].append(pos_start)
            lines[i].append(length)
            good_lines[scaf_name].append(lines[i])
    data_fin = {}
    for name_of_scaffold in names_of_scaffolds:
        if name_of_scaffold in good_lines.keys():
            res = good_lines[name_of_scaffold]

            # leaving only necessary information
            data_fin_sc = []
            for x in res:
                if x[1].startswith('NODE'):
                    data_fin_sc.append([x[1], int(x[-2]), int(x[-2]) + int(x[-1]), x[4], int(x[2]), int(x[2]) + int(x[-1]), int(x[-1])])
            data_fin[name_of_scaffold] = data_fin_sc
    return data_fin



In [ ]:
def mushroom_names():
    with open("tba_ref_ordered.maf") as fin:
        fin.seek(33)
        data = fin.read(1000000)

    names_of_mushrooms = []
    find = re.compile("^(.*)\..*")
    for line in data.splitlines():
        if len(line) > 2:
            m = re.match(find, line.split()[1])
            if type(m) == re.Match: 
                names_of_mushrooms.append(m.group(1))
    del names_of_mushrooms[0]
    del names_of_mushrooms[0]
    real_names_of_mushrooms = []
    for x in names_of_mushrooms:
        if not x.startswith('score'):
            if x not in real_names_of_mushrooms:
                real_names_of_mushrooms.append(x)
    real_names_of_mushrooms.remove('ref')
    return real_names_of_mushrooms

mushroom_names = mushroom_names()





In [ ]:
mushroom_names = ['X-9_S78', 'X-12_S79', 'X-17_S80', 'X-27_S82', 'X-30_S83',
                  'X-69_S85', 'S1_contigs', 'S2_contigs', 'S4_contigs',
                 '14-01_S62', '14-101_S73', '14-102_S74', '14-104_S75',
                  '14-112_S77', '14-11_S63', '14-24', '14-25_S64',
                  '14-29_S65', '14-31_S66', '14-32_S67', '14-34_S68',
                  '14-41', '14-67_S84', '14-70_S69', '14-85_S70',
                  '14-89_S71', '14-90_S72', '14-96', 'A10_contigs',
                  'A3_contigs', 'B3_contigs', 'B6_contigs',
                  'A7_contigs', 'B1_contigs', 'B5_contigs', 'A5_contigs',
                  'A4_contigs', 'A13_contigs','K1_contigs', 'K3_contigs', 'K4_contigs',
                  'M2_contigs', 'M3_contigs', 'K1_contigs', 'K3_contigs', 'K4_contigs',
                  'FL_contigs', 'M1_contigs', 'X-21_S81', 'X-44', '15-14_S76', 'A8_contigs',
                  'M4_contigs', 'M5_contigs', 'M6_contigs', 'Mi1_contigs', 'Mos_contigs']

# for m in mushroom_names:
#     filename = 'ref_' + m + '.maf'
#     with open(filename) as fin:
#         fin.seek(0) #395
#         data = fin.read()
#     data = data.split('\n')

#!  'A8_contigs'



In [ ]:
inv = {}

In [ ]:
#mushroom_names = ['X-12_S79']
for mushroom_name in mushroom_names:
    print(mushroom_name)
    data_fin = get_data(mushroom_name)
    inv[mushroom_name] = inversion_search(data_fin)

In [ ]:
len(mushroom_names)

In [ ]:
inv = {}

In [ ]:
mushroom_names_ad = ['X-21_S81', 'X-44', '15-14_S76', 'A8_contigs', 'M4_contigs', 'M5_contigs', 'M6_contigs',
                  'Mi1_contigs', 'Mos_contigs']

In [ ]:
for mushroom_name in mushroom_names_ad:
    print(mushroom_name)
    data_fin = get_data(mushroom_name)
    inv[mushroom_name] = inversion_search(data_fin)

In [ ]:
len(inv.keys())

# Writing data

In [ ]:
inv_df = pd.DataFrame(inv)
#inv_df_3.to_csv('inv_df_pairwise_all.csv')


In [ ]:
origin = {}
for name in mushroom_names:
    if name[0] in ['A', 'B', '1']:
        origin[name] = 'a'
    else:
        origin[name] = 'r'
        
        
origin['s1514'] = 'a' #fl
origin['FL'] = 'a' #fl
origin['Mi1'] = 'a'

# Reading data

In [ ]:
inv_df = pd.read_csv('inv_df_pairwise_all.csv',  error_bad_lines=False)
inv_df = inv_df.rename({'Unnamed: 0': 'sample'}, axis='columns')


In [ ]:
inv_df.set_index('sample', inplace=True)
dict_inv = inv_df.to_dict()


In [ ]:
fin_dict = {}
for m in dict_inv.keys():
    fin_dict[m] = {}
    for sc in dict_inv[m].keys():
        if type(dict_inv[m][sc]) is not float:
            if len(dict_inv[m][sc]) == 2:
                fin_dict[m][sc] = []
            else:
                fin_dict[m][sc] = []
                for x in dict_inv[m][sc][3:-3].split('], ['):
                    res = []
                    for y in x.split(', '):
                        if y.startswith("'"):
                            res.append(y[1:-1])
                        elif y[-1] == ']':
                            res.append(y[:-1])
                        elif y.startswith("["):
                            res.append(y[2:-1])
                        else:
                            res.append(y)
                    fin_dict[m][sc].append(res)

In [ ]:
all_inv = {}
for sc in names_of_scaffolds:
    all_inv[sc] = {}
    
    
for m in fin_dict.keys():
    for sc in fin_dict[m].keys():
        for x in fin_dict[m][sc]: # every inv
            appended = 0 
            for y in all_inv[sc].keys(): # every inversion already in the list
                if abs(int(x[2]) - y) < 100:
                    if m not in all_inv[sc][y]:
                        all_inv[sc][y].append(m) 
                    appended = 1
            if appended == 0:
                all_inv[sc][int(x[2])] = [m]

In [ ]:
len(fin_dict.keys())

In [ ]:
for key1 in all_inv['scaffold_10']:
    for key2 in all_inv['scaffold_10']:
        if key1 != key2:
            if abs(key1 - key2) < 100:
                print(key1, key2)

In [ ]:
all_inv_listed = {}
for sc in all_inv.keys():
    for c in all_inv[sc].keys():
        all_inv_listed[sc + '_' + str(c)] = all_inv[sc][c]

In [ ]:
all_inv_freq = {}
for k in all_inv_listed.keys():
    all_inv_freq[k] = len(all_inv_listed[k])

In [ ]:
plt.hist(all_inv_freq.values(), bins=np.arange(0,16)+0.5, color = "#b0c40c")
plt.title('Frequencies of inversion breakpoints')
plt.xlabel('Number of organisms')
plt.ylabel('Number of inversion breakpoints')
plt.show()

In [ ]:
origin = {}
for name in fin_dict.keys():
    if name[0] in ['A', 'B', '1']:
        origin[name] = 'a'
    else:
        origin[name] = 'r'
        
        
origin['s1514'] = 'a'
origin['FL_contigs'] = 'a'
origin['Mi1'] = 'a'

In [ ]:
len(fin_dict.keys())

In [ ]:
am_inv_freq = {}
rus_inv_freq = {}
for k in all_inv_listed.keys():
    rus_inv_freq[k] = 0
    am_inv_freq[k] = 0
    for x in all_inv_listed[k]:
        if origin[x] == 'r':
            rus_inv_freq[k] += 1
        else:
            am_inv_freq[k] += 1
am_inv_freq = {k: v for k, v in am_inv_freq.items() if v != 0}
rus_inv_freq = {k: v for k, v in rus_inv_freq.items() if v != 0}

In [ ]:
plt.hist(rus_inv_freq.values(), bins=np.arange(0,16)+0.5, color = "#b0c40c")
plt.title('Frequencies of inversion breakpoints in Russian population')
plt.xlabel('Number of organisms')
plt.ylabel('Number of inversion breakpoints')
plt.show()

In [ ]:
plt.hist(am_inv_freq.values(), bins=np.arange(0,22)+0.5, color = "#b0c40c")
plt.title('Frequencies of inversion breakpoints in American population')
plt.xlabel('Number of organisms')
plt.ylabel('Number of inversion breakpoints')
plt.show()

# checking if we know at least some lengths

In [ ]:
fin_dict_paired = {}
for m in fin_dict.keys():
    fin_dict_paired[m] = {}
    for sc in fin_dict[m].keys():
        fin_dict_paired[m][sc] = []
        for i in range(len(fin_dict[m][sc])//2):
            res = [fin_dict[m][sc][i*2], fin_dict[m][sc][i*2+1]]
            fin_dict_paired[m][sc].append(res)

In [ ]:
lengths = {}
num_inv = 0

for m in fin_dict_paired.keys():
    lengths[m] = []
    for sc in fin_dict_paired[m].keys():     
        used = []
        for bp1 in fin_dict_paired[m][sc]:
            for bp2 in fin_dict_paired[m][sc]:
                if bp1 != bp2:
                    if bp1[0][0] == bp2[0][0]:
                        if bp1[0][3] != bp2[0][3]:
                            if bp1 not in used and bp2 not in used:
                                used.append(bp1)
                                used.append(bp2)
                                length = abs(int(bp1[0][2]) - int(bp2[0][2]))
                                lengths[m].append(length)
                                num_inv += 1

In [ ]:
lengths_list = []
lengths_r = []
lengths_am = []

for m in lengths.keys():
    if len(lengths[m]) != 0:
        for x in lengths[m]:
            lengths_list.append(x)
            if origin[m] == 'r':
                lengths_r.append(x)
            else:
                lengths_am.append(x)

In [ ]:
np.mean(lengths_list)

In [ ]:
np.median(lengths_list)

In [ ]:
len(lengths_list)

In [ ]:
def plot_loghist(x, bins):
  hist, bins = np.histogram(x, bins=bins)
  logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
  plt.hist(x, bins=logbins,  color = "#b0c40c")
  plt.xscale('log')

plot_loghist(lengths_list, 10)



plt.title("Distribution of inversion lengths")
plt.ylabel('Number of inversions')
plt.xlabel('Length, bp')
plt.yticks(range(0, 15, 1))
plt.show()

In [ ]:
plot_loghist(lengths_r, 10)
plt.title("Distribution of inversion lengths, Russian samples")
plt.ylabel('Number of inversions')
plt.xlabel('Length, bp')
plt.show()

In [ ]:
plot_loghist(lengths_am, 10)
plt.title("Distribution of inversion lengths, American samples")
plt.ylabel('Number of inversions')
plt.xlabel('Length, bp')
plt.show()

In [ ]:
num_rus = 0
for m in origin.keys():
    if origin[m] == 'r':
        num_rus +=1
print(num_rus)

# Only real inversions

In [ ]:
fin_dict_paired = {}
for m in fin_dict.keys():
    fin_dict_paired[m] = {}
    for sc in fin_dict[m].keys():
        fin_dict_paired[m][sc] = []
        for i in range(len(fin_dict[m][sc])//2):
            fin_dict_paired[m][sc].append([fin_dict[m][sc][i*2],  fin_dict[m][sc][i*2+1]])

In [ ]:
real_inv = {}
cs = []
for m in fin_dict_paired.keys():
    real_inv[m] = {}
    data = get_data(m)
    for sc in fin_dict_paired[m].keys():
        real_inv[m][sc] = []
        for x in fin_dict_paired[m][sc]:
            coord = np.mean([int(x[0][2]), int(x[1][1])])
            c = 0
            for line in data[sc]:
                #if abs(line[1] - coord) < 30 or abs(line[2] - coord) < 30:
                if line[1] < coord:
                    if line[2] > coord:
                        c += 1
            cs.append(c)
            if c < 3:
                real_inv[m][sc].append(x)
                

In [ ]:
def plot_loghist(x, bins):
  hist, bins = np.histogram(x, bins=bins)
  logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
  plt.hist(x, bins=logbins,  color = "#b0c40c")
  plt.xscale('log')

cs_1 = [x for x in cs if x != 0]
plot_loghist(cs_1, 10)
plt.show()

In [ ]:
real_inv

In [ ]:
for x in cs:
    if x < 2:
        print(x)

In [ ]:
pd.DataFrame.from_dict(real_inv)

In [ ]:
all_inv = {}
for sc in names_of_scaffolds:
    all_inv[sc] = {}
    
    
for m in real_inv.keys():
    for sc in real_inv[m].keys():
        for x in real_inv[m][sc]: # every inv
            appended = 0 
            for y in all_inv[sc].keys(): # every inversion already in the list
                if abs(int(x[0][2]) - y) < 100:
                    if m not in all_inv[sc][y]:
                        all_inv[sc][y].append(m) 
                    appended = 1
            if appended == 0:
                all_inv[sc][np.mean([int(x[0][2]), int(x[1][1])])] = [m]

In [ ]:

all_inv_listed = {}
for sc in all_inv.keys():
    for c in all_inv[sc].keys():
        all_inv_listed[sc + '_' + str(c)] = all_inv[sc][c]

all_inv_freq = {}
for k in all_inv_listed.keys():
    all_inv_freq[k] = len(all_inv_listed[k])

plt.hist(all_inv_freq.values(), bins=np.arange(0,16)+0.5, color = "#b0c40c")
plt.title('Frequencies of inversion breakpoints')
plt.xlabel('Number of organisms')
plt.ylabel('Number of inversion breakpoints')
plt.show()

In [ ]:
all_inv_listed

Strange but maybe

In [ ]:
real_inv

In [ ]:

def check_others():
    no_inv = {}
    for m1 in real_inv.keys():
        no_inv[m1] = {}
        data = get_data(m1)
        for sc in real_inv[m1].keys():
            no_inv[m1][sc] = []
            for m in real_inv.keys():
                if sc in real_inv[m].keys():
                    if real_inv[m][sc] != []:
                        #print(real_inv[m][sc][0][0])
                        coord1 = min(int(real_inv[m][sc][0][0][2]), int(real_inv[m][sc][0][1][1]))
                        coord2 = max(int(real_inv[m][sc][0][0][2]), int(real_inv[m][sc][0][1][1]))
                        no_inv_fl = 0
                        for line in data[sc]:
                            if int(line[1]) < coord1 and int(line[2]) > coord2:
                                no_inv_fl = 1
                        if no_inv_fl == 0:
                            no_inv[m1][sc].append(real_inv[m][sc])
    return no_inv
                    
                    

In [ ]:
no_inv = check_others()

In [ ]:
no_inv_listed = {}
for m in no_inv.keys():
    for sc in no_inv[m].keys():
        for inv in no_inv[m][sc]:
            coord = np.mean([int(inv[0][0][2]), int(inv[0][1][1])])
            if sc + '_' + str(coord) in no_inv_listed.keys():
                no_inv_listed[sc + '_' + str(coord)].append(m)
            else: 
                no_inv_listed[sc + '_' + str(coord)] = [m]

In [ ]:
no_inv_listed

In [ ]:
all_inv_listed

In [ ]:
def rear_inv(datum):
    inv = {}
    for sc in datum.keys():
        data = datum[sc]
        inv[sc] = []
        used_bps = []
        for i in range(len(data)):
            for j in range(len(data)):
                if data[i][0] == data[j][0]:
                    if data[i][3] != data[j][3]:
                        #print('inv')
                        if (abs(data[i][1] - data[j][2]) < 30) or (abs(data[j][1] - data[i][2]) < 30):
                            if (data[i][-1] > 30) and (data[j][-1] > 30):
                                if (data[i][1] < data[j][1]):
                                    if data[i] not in used_bps and data[j] not in used_bps:
                                        already_in_list = 0
                                        for x in inv[sc]:
                                            #print(x, data[i])
                                            if abs(x[0][1] - data[i][1]) < 30 or abs(x[1][1] - data[j][1]) < 30:
                                                already_in_list = 1
                                        if already_in_list == 0:
                                            inv[sc].append([data[i], data[j]])
                                        used_bps.append(data[i])
                                        used_bps.append(data[j])
    return inv

In [ ]:
rear_inv_d = {}
for mushroom_name in mushroom_names:
    print(mushroom_name)
    data_fin = get_data(mushroom_name)
    rear_inv_d[mushroom_name] = rear_inv(data_fin)

In [ ]:
rear_df = pd.DataFrame(rear_inv_d)
rear_df.to_csv('rear_df_pairwise_all.csv')


In [ ]:
rear_df = pd.read_csv('rear_df_pairwise_all.csv',  error_bad_lines=False)
rear_df = rear_df.rename({'Unnamed: 0': 'sample'}, axis='columns')


In [ ]:
inv_df.set_index('sample', inplace=True)
dict_inv = inv_df.to_dict()


fin_dict = {}
for m in dict_inv.keys():
    fin_dict[m] = {}
    for sc in dict_inv[m].keys():
        if type(dict_inv[m][sc]) is not float:
            if len(dict_inv[m][sc]) == 2:
                fin_dict[m][sc] = []
            else:
                fin_dict[m][sc] = []
                for x in dict_inv[m][sc][3:-3].split('], ['):
                    res = []
                    for y in x.split(', '):
                        if y.startswith("'"):
                            res.append(y[1:-1])
                        elif y[-1] == ']':
                            res.append(y[:-1])
                        elif y.startswith("["):
                            res.append(y[2:-1])
                        else:
                            res.append(y)
                    fin_dict[m][sc].append(res)

In [ ]:
real_inv.keys()

In [ ]:
data = get_data('X-9_S78')
coord = 103082
for line in data['scaffold_3']:
    if abs(line[1] - coord) < 100:
        print(line)
    if abs(line[2] - coord) < 100:
        print(line)    

In [ ]:
data = get_data('14-31_S66')
coord = 103082
for line in data['scaffold_3']:
    if abs(line[1] - coord) < 1000:
        print(line)
    if abs(line[2] - coord) < 1000:
        print(line) 

In [ ]:
data = get_data('FL_contigs')
coord = 318998
for line in data['scaffold_7']:
    if abs(line[1] - coord) < 100:
        print(line)
    if abs(line[2] - coord) < 100:
        print(line)    

Good breakpoint (inside CDS)

In [ ]:
data = get_data('14-24')
coord = 592750
for line in data['scaffold_2']:
    if abs(line[1] - coord) < 100:
        print(line)
    if abs(line[2] - coord) < 100:
        print(line)    

In [ ]:
data = get_data('M4_contigs')
coord = 2802673
for line in data['scaffold_4']:
    if abs(line[1] - coord) < 100:
        print(line)
    if abs(line[2] - coord) < 100:
        print(line)  

In [ ]:
data = get_data('A10_contigs')
coord = 2802673
for line in data['scaffold_4']:
    if abs(line[1] - coord) < 1000:
        print(line)
    if abs(line[2] - coord) < 1000:
        print(line)  

Questionable breakpoint

In [ ]:
data = get_data('X-69_S85')
coord = 1108053
for line in data['scaffold_7']:
    if abs(line[1] - coord) < 100:
        print(line)
    if abs(line[2] - coord) < 100:
        print(line)  

Good beakpoint

In [ ]:
data = get_data('A5_contigs')
coord = 194921
for line in data['scaffold_6']:
    if abs(line[1] - coord) < 100:
        print(line)
    if abs(line[2] - coord) < 100:
        print(line)  

# Distribution by scaffold

In [ ]:
with open ("scaffolds_len_ref.txt") as fin:
    data = fin.read()
data = data.splitlines()

In [ ]:
scaf_len = {}
for i in range(len(data)//2):
    scaf_len[data[2*i][1:]] = int(data[2*i+1])

In [ ]:
scaf_len

In [ ]:
scaff_counts = {}
for sc in all_inv.keys():
    scaff_counts[sc] = 0
    for x in all_inv[sc]:
        scaff_counts[sc] += 1

In [ ]:
plt.bar(range(1,26), list(scaff_counts.values()))
plt.show()

In [ ]:
x = list(scaf_len.values())
y = list(scaff_counts.values())
correlation_matrix = np.corrcoef(x, y)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

In [ ]:
r_squared

In [ ]:

plt.scatter(x,  y, s=6)
plt.xlabel('Length of scaffold, bp')
plt.ylabel('Number of breakpoints')
plt.title('The correlation between the number of breakpoints and the length of the scaffold')
plt.show()

In [ ]:
scaff_counts.values()

In [ ]:
sum(y)

# Inversions breaking genes

In [ ]:
in_file = "Schco3.filtered_proteins.FilteredModels1.gff3"
examiner = GFFExaminer()
in_handle = open(in_file)
pprint.pprint(examiner.available_limits(in_handle))
in_handle.close()

In [ ]:
genes = {}
for scaffold_name in names_of_scaffolds:
    sc = 'scaffold_' + re.findall('[0-9]+', scaffold_name)[0]
    limit_info = dict(gff_id=[sc], gff_type=['gene'])
    gene_locs = []

    in_handle = open(in_file)
    for rec in GFF.parse(in_handle, limit_info=limit_info):
        feature = rec.features
        for seq in feature:
            gene_locs.append(seq.location)
    in_handle.close()
    gene_coords = []
    for loc in gene_locs:
        gene_coords.append([int(loc.start), int(loc.end)])
    genes[scaffold_name] = gene_coords

In [ ]:
inv_breaking_genes = []
for sc in all_inv.keys():
    for c in all_inv[sc].keys():
        for gene in genes[sc]:
            if gene[0] < c and gene[1] > c:
                inv_breaking_genes.append(all_inv[sc][c])

In [ ]:
len(inv_breaking_genes)

In [ ]:
inv_breaking_genes

In [ ]:
num = 0
for sc in all_inv.keys():
    num += len(all_inv[sc])
print(num)

In [ ]:
len(all_

In [ ]:
num/753

calculating which part of genome is genes

In [ ]:
gene_length = 0
for sc in names_of_scaffolds:
        for gene in genes[sc]:
            gene_length += gene[1] - gene[0]


In [ ]:
25510111/35000000

In [ ]:
gene_length

In [ ]:
genes

In [ ]:
sc_length = {}
with open("tba_ref_ordered.maf") as fin:
    fin.seek(33)
    data = fin.read()
data = data.split('\n')
for line in data:
    ar = line.split(' ')
    if len(ar) > 2:
        if ar[1].startswith('ref'):
            if ar[1] not in sc_length.keys():
                sc_length[ar[1]] = int(ar[-2])

In [ ]:
total_length = 0
for sc in sc_length.keys():
    total_length += sc_length[sc]

In [ ]:
total_length

In [ ]:
gene_length/total_length

# Inversions breaking genes

In [ ]:
genes = {}
for scaffold_name in names_of_scaffolds:
    sc = 'scaffold_' + re.findall('[0-9]+', scaffold_name)[0]
    limit_info = dict(gff_id=[sc], gff_type=['gene'])
    gene_locs = []

    in_handle = open(in_file)
    for rec in GFF.parse(in_handle, limit_info=limit_info):
        feature = rec.features
        for seq in feature:
            gene_locs.append(seq.location)
    in_handle.close()
    gene_coords = []
    for loc in gene_locs:
        gene_coords.append([int(loc.start), int(loc.end)])
    genes[scaffold_name] = gene_coords

In [ ]:
inv_breaking_genes = []
for sc in all_inv.keys():
    for c in all_inv[sc].keys():
        for gene in genes[sc]:
            if gene[0] < c and gene[1] > c:
                inv_breaking_genes.append(all_inv[sc][c])

In [ ]:
len(inv_breaking_genes)

In [ ]:
121/753

In [ ]:
 am_inv_freq

# Most common inversions

Only in Russian mushrooms

In [ ]:
for inv in rus_inv_freq.keys():
        if rus_inv_freq[inv] > 10:
            print(inv, all_inv_listed[inv])

In [ ]:
for inv in all_inv_listed.keys():
    if inv not in am_inv_freq.keys():
        if rus_inv_freq[inv] > 1:
            print(inv, all_inv_listed[inv])

In [ ]:
origin


In [ ]:
num = {}
for m in fin_dict.keys():
    num[m] = 0
    for sc in fin_dict[m].keys():
        num[m] += len(fin_dict[m][sc])

In [ ]:
num_rus = []
num_am = []
for m in num.keys():
    if origin[m] == 'r':
        num_rus.append(num[m])
    else:
        num_am.append(num[m])

In [ ]:
plt.boxplot([num_rus, num_am], labels = ['Russia', 'USA'])
plt.ylabel('Number of inversion breakpoints')
plt.show()

In [ ]:
np.mean(num_rus)

In [ ]:
np.mean(num_am)

In [ ]:
num['M3_contigs'] =25

In [ ]:
num

In [ ]:
fin_dict['X-12_S79']

In [ ]:
np.sum(list(rus_inv_freq.values()))/15

In [ ]:
np.sum(list(am_inv_freq.values()))/30

In [ ]:
num_rus = 0
for m in origin.keys():
    if origin[m] == 'r':
        num_rus +=1
print(num_rus)

In [ ]:
data = get_data('A10_contigs')

In [ ]:
num = {}
for sc in names_of_scaffolds:
    num[sc] = 0
    for line in data[sc]:
        if line[0].startswith('NODE_49_'):
            num[sc] +=1

In [ ]:
num

In [ ]:
for line in data['scaffold_3']:
    if abs(line[1] - 181781) < 100:
        print(line)
    if abs(line[2] - 181781) < 100:
        print(line)
    

In [ ]:
data = get_data('X-17_S80')
for line in data['scaffold_3']:
    if abs(line[1] - 181781) < 100:
        print(line)

In [ ]:
data = get_data( '14-41')
for line in data['scaffold_3']:
    if abs(line[1] - 181781) < 100:
        print(line)
    if abs(line[2] - 181781) < 100:

In [ ]:
for line in data['scaffold_3']:
    if abs(line[1] - 181781) < 100:
        print(line)
    if abs(line[2] - 181781) < 100:
        print(line)

In [ ]:
len(fin_dict.keys())